# A testing ground for pyPIPS functionality

In [ ]:
%load_ext autoreload
%autoreload 2
import itertools
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
import pyPIPS.radarmodule as radar
import pyPIPS.timemodule as tm
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
import pyart
import cartopy
import cartopy.crs as ccrs
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
min_diameter = pp.parsivel_parameters['min_diameter_bins_mm']
max_diameter = pp.parsivel_parameters['max_diameter_bins_mm']
bin_width = max_diameter - min_diameter
avg_diameter = pp.parsivel_parameters['avg_diameter_bins_mm']
min_fall_bins = pp.parsivel_parameters['min_fallspeed_bins_mps']
max_fall_bins = pp.parsivel_parameters['max_fallspeed_bins_mps']
avg_fall_bins = pp.parsivel_parameters['avg_fallspeed_bins_mps']

In [ ]:
# Dynamically import the case configuration file
case_config_path = '/Users/dawson29/Projects/pyPIPS/configs/PERiLS_IOP2_2023_60s.py'

utils.log("Case config file is {}".format(case_config_path))
config = utils.import_all_from(case_config_path)
try:
    config = utils.import_all_from(case_config_path)
    utils.log("Successfully imported case configuration parameters!")
except Exception:
    utils.fatal(
        "Unable to import case configuration parameters! Aborting!")


In [ ]:
# Extract needed lists and variables from PIPS_IO_dict configuration dictionary
dataset_name = config.PIPS_IO_dict.get('dataset_name', None)
deployment_names = config.PIPS_IO_dict.get('deployment_names', None)
PIPS_dir = config.PIPS_IO_dict.get('PIPS_dir', None)
plot_dir = config.PIPS_IO_dict.get('plot_dir', None)
PIPS_types = config.PIPS_IO_dict.get('PIPS_types', None)
PIPS_names = config.PIPS_IO_dict.get('PIPS_names', None)
PIPS_filenames = config.PIPS_IO_dict.get('PIPS_filenames', None)
start_times = config.PIPS_IO_dict.get('start_times', [None] * len(PIPS_names))
end_times = config.PIPS_IO_dict.get('end_times', [None] * len(PIPS_names))
geo_locs = config.PIPS_IO_dict.get('geo_locs', [None] * len(PIPS_names))
requested_interval = config.PIPS_IO_dict.get('requested_interval', 10.)

# Extract needed lists and variables from the radar_dict configuration dictionary
comp_radar = config.radar_config_dict.get('comp_radar', False)
clean_radar = config.radar_config_dict.get('comp_radar', False)
calc_dualpol = config.radar_config_dict.get('calc_dualpol', False)
radar_name = config.radar_config_dict.get('radar_name', None)
radar_type = config.radar_config_dict.get('radar_type', None)
radar_dir = config.radar_config_dict.get('radar_dir', None)
radar_fname_pattern = config.radar_config_dict.get('radar_fname_pattern', None)
# Add the input filename tag to the pattern if needed
radar_input_tag = 'filt_retr'

if radar_input_tag:
    radar_fname_pattern = radar_fname_pattern.replace('.', '_{}.'.format(radar_input_tag))
field_names = config.radar_config_dict.get('field_names', ['REF'])
if not calc_dualpol:
    field_names = ['REF']

el_req = config.radar_config_dict.get('el_req', 0.5)
radar_start_timestamp = config.radar_config_dict.get('radar_start_timestamp', None)
radar_end_timestamp = config.radar_config_dict.get('radar_end_timestamp', None)
scatt_dir = config.radar_config_dict.get('scatt_dir', None)
wavelength = config.radar_config_dict.get('wavelength', 10.7)

In [ ]:
# Get a list of the combined parsivel netCDF data files that are present in the PIPS directory
# TODO: change this logic to actually use the files listed in the config file
parsivel_combined_filenames = [
    'parsivel_combined_{}_{}_{:d}s.nc'.format(deployment_name, PIPS_name, int(requested_interval))
    for deployment_name, PIPS_name in zip(deployment_names, PIPS_names)]
parsivel_combined_filelist = [os.path.join(PIPS_dir, pcf) for pcf in parsivel_combined_filenames]

In [ ]:
parsivel_combined_filelist

In [ ]:
new_pc_list = parsivel_combined_filelist # [pcf for pcf in parsivel_combined_filelist if 'PIPS2B' not in pcf]
new_pc_list

In [ ]:
radar_paths = glob.glob(radar_dir + '/*{}*_{}.nc'.format(radar_name, radar_input_tag))
# Then find only those between the requested times
radar_path_dict = radar.get_radar_paths_between_times(radar_paths, radar_start_timestamp,
                                                      radar_end_timestamp, radar_type=radar_type,
                                                      fname_format=radar_fname_pattern)

In [ ]:
radar_path_dict

In [ ]:
# Read first sweep in to get radar location
first_sweep_list = radar.readCFRadial_pyART(el_req, radar_path_dict['rad_path_list'][0],
                                            compute_kdp=False)
first_sweep = first_sweep_list[0]
rlat = first_sweep.latitude['data'][0]
rlon = first_sweep.longitude['data'][0]
ralt = first_sweep.altitude['data'][0]

In [ ]:
geo_loc_dict = {}
rad_loc_dict = {}
PIPS_names = []
PIPS_ds_dict = {}
for index, parsivel_combined_file in enumerate(new_pc_list):
    print("Reading {}".format(parsivel_combined_file))
    parsivel_combined_ds = xr.load_dataset(parsivel_combined_file)
    PIPS_name = parsivel_combined_ds.probe_name
    PIPS_ds_dict[PIPS_name] = parsivel_combined_ds
    PIPS_names.append(PIPS_name)
    deployment_name = parsivel_combined_ds.deployment_name
#     image_dir = os.path.join(radar_ppi_image_dir, deployment_name)
#     if not os.path.exists(image_dir):
#         os.makedirs(image_dir)
    geo_loc_str = parsivel_combined_ds.location
    geo_loc = list(map(float, geo_loc_str.strip('()').split(',')))
    geo_loc_dict[PIPS_name] = geo_loc
    rad_loc = radar.get_PIPS_loc_relative_to_radar(geo_loc, rlat, rlon, ralt)
    rad_loc_dict[PIPS_name] = rad_loc

In [ ]:
PIPS_ds = PIPS_ds_dict['PIPS2A']
PIPS_ds_start_time = PIPS_ds.time[0].values
PIPS_ds_end_time = PIPS_ds.time[-1].values

print(PIPS_ds_start_time, PIPS_ds_end_time)

In [ ]:
PIPS_x = [rad_loc_dict[PIPS_name][0] for PIPS_name in PIPS_names]
PIPS_y = [rad_loc_dict[PIPS_name][1] for PIPS_name in PIPS_names]

# buffer zone in meters surrounding PIPS for radar plot
buffer_x = 20000.
buffer_y = 20000.

xmin = min(PIPS_x) - buffer_x
xmax = max(PIPS_x) + buffer_x
ymin = min(PIPS_y) - buffer_y
ymax = max(PIPS_y) + buffer_y

bounds = [xmin, xmax, ymin, ymax]

In [ ]:
from pyPIPS.radarmodule import *

def mtokm(val,pos):
    """Convert m to km for formatting axes tick labels"""
    val=val/1000.0
    return '%i' % val

def plotsweep_pyART(radar_obj, sweeptime, PIPS_names, PIPS_geo_locs, PIPS_rad_locs, radar_fields,
                    PIPS_fields=None, bounds=[-20000., 20000., -20000., 20000.], plot_filtered=False):
    """
    Plots an individual radar sweep, with an option to overlay a map.  This version uses pyART
    routines and assumes the radar sweep has been read in using readCFRadial_pyART.
    """

    display = pyart.graph.RadarMapDisplay(radar_obj)
    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])

    if False:
        PIPS_x = [PIPS_rad_loc[0] for PIPS_rad_loc in PIPS_rad_locs]
        PIPS_y = [PIPS_rad_loc[1] for PIPS_rad_loc in PIPS_rad_locs]
        xmin = min(PIPS_x) + bounds[0] * 1000.
        xmax = max(PIPS_x) + bounds[1] * 1000.
        ymin = min(PIPS_y) + bounds[2] * 1000.
        ymax = max(PIPS_y) + bounds[3] * 1000.
    else:
        xmin = bounds[0]
        xmax = bounds[1]
        ymin = bounds[2]
        ymax = bounds[3]

    figlist = []
    axlist = []

    # Try to match up each field with what is in the file
    # TODO: split this part off into a separate function
    fields_to_plot = []
    for field in radar_fields:
        # Reflectivity
        if field in REF_aliases:
            field_to_plot = get_field_to_plot(radar_obj, REF_aliases)
        elif field in ZDR_aliases:
            field_to_plot = get_field_to_plot(radar_obj, ZDR_aliases)
        elif field in KDP_aliases:
            field_to_plot = get_field_to_plot(radar_obj, KDP_aliases)
        elif field in RHV_aliases:
            field_to_plot = get_field_to_plot(radar_obj, RHV_aliases)
        elif field in VR_aliases:
            field_to_plot = get_field_to_plot(radar_obj, VR_aliases)
        else:
            # Try to find the field "as is" in the file
            try:
                field_arr = radar_obj.fields[field]
                field_to_plot = (field, field_arr)
            except KeyError:
                field_to_plot = None
        if field_to_plot:
            fields_to_plot.append(field_to_plot[0])

        if plot_filtered:
            if field in REF_aliases:
                field_to_plot = get_field_to_plot(radar_obj, REF_aliases, tag='_filtered')
            elif field in ZDR_aliases:
                field_to_plot = get_field_to_plot(radar_obj, ZDR_aliases, tag='_filtered')
            elif field in RHV_aliases:
                field_to_plot = get_field_to_plot(radar_obj, RHV_aliases, tag='_filtered')
            if field_to_plot:
                fields_to_plot.append(field_to_plot[0])

    for field in fields_to_plot:
        fig = plt.figure(figsize=(10, 8))
        titlestringfmt = 'Radar name: {}; Field: {}; Time: {}; elevation: {:2.1f}'
        titlestring = titlestringfmt.format(radar_obj.metadata['instrument_name'], field,
                                            sweeptime.strftime(tm.timefmt3),
                                            radar_obj.elevation['data'][0])

        field_to_match = field
        # DTD: find a better way to do this
        retr_aliases = [retr_alias for aliases in retr_alias_list for retr_alias in aliases]
        for field_trial in retr_aliases:
            if field_trial in field:
                field_to_match = field_trial
                break
        field_to_match = field_to_match.replace('_filtered', '')
        field_plot_params = radar_plot_param_matching[field_to_match]

        # TODO: add colorbar label levels, other arguments
        display.plot_ppi_map(field, 0, title_flag=False, cmap=field_plot_params['cmap'],
                             vmin=field_plot_params['clevels'][0],
                             vmax=field_plot_params['clevels'][-1], colorbar_label='', # ax=ax,
                             resolution='10m', projection=projection, fig=fig,
                             lat_lines=np.arange(30, 46, 0.1), lon_lines=np.arange(-110, -75, 0.1),
                             raster=True)
        display.ax.set_title(titlestring, fontsize=10)
        # Overlay locations of the PIPS
        for PIPS_geo_loc, PIPS_name in zip(PIPS_geo_locs, PIPS_names):
            display.plot_point(PIPS_geo_loc[1], PIPS_geo_loc[0], 'r*', ms=10, alpha=0.5,
                               label_text=PIPS_name)

        # Overlay roads
        display.ax.add_feature(cartopy.feature.NaturalEarthFeature('cultural', 'roads', '10m'),
                               facecolor='none', edgecolor='black', alpha=.3)
            
        display.ax.set_extent([xmin, xmax, ymin, ymax], crs=projection)

        figlist.append(fig)
        axlist.append(display.ax)

    return figlist, axlist, fields_to_plot


def plotsweep_pcolor(radar_obj, radar_fields, sweeptime, PIPS_names=None, PIPS_rad_loc_dict=None, 
                     PIPS_fields=None, bounds=[-20000., 20000., -20000., 20000.], plot_filtered=False,
                     norm=None, cbarlabel=None, axestickintv=10000.):
    
    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])
    
    # Find nice text label positions for the PIPS overlays
    if PIPS_names is not None and PIPS_rad_loc_dict is not None:
        PIPS_x_dict = {PIPS_name: PIPS_rad_loc_dict[PIPS_name][0] for PIPS_name in PIPS_names}
        PIPS_y_dict = {PIPS_name: PIPS_rad_loc_dict[PIPS_name][1] for PIPS_name in PIPS_names}
        text_x_dict = {PIPS_name: PIPS_x_dict[PIPS_name] + 500. for PIPS_name in PIPS_names}
        text_y_dict = {PIPS_name: PIPS_y_dict[PIPS_name] + 500. for PIPS_name in PIPS_names}
        
        for PIPS_pair in itertools.combinations(PIPS_names, 2):
            if (np.abs(text_x_dict[PIPS_pair[0]] - text_x_dict[PIPS_pair[1]]) < 500. and
                np.abs(text_y_dict[PIPS_pair[0]] - text_y_dict[PIPS_pair[1]]) < 500.):
                text_y_dict[PIPS_pair[0]] -= 1000.
    
    xplt, yplt, _ = radar_obj.get_gate_x_y_z(0, edges=True)
    xplt = xplt.squeeze()
    yplt = yplt.squeeze()
    
    xmin = bounds[0]
    xmax = bounds[1]
    ymin = bounds[2]
    ymax = bounds[3]
    
    figlist = []
    axlist = []

    # Try to match up each field with what is in the file
    # TODO: split this part off into a separate function
    fields_to_plot = []
    for field in radar_fields:
        # Reflectivity
        if field in REF_aliases:
            field_to_plot = get_field_to_plot(radar_obj, REF_aliases)
        elif field in ZDR_aliases:
            field_to_plot = get_field_to_plot(radar_obj, ZDR_aliases)
        elif field in KDP_aliases:
            field_to_plot = get_field_to_plot(radar_obj, KDP_aliases)
        elif field in RHV_aliases:
            field_to_plot = get_field_to_plot(radar_obj, RHV_aliases)
        elif field in VR_aliases:
            field_to_plot = get_field_to_plot(radar_obj, VR_aliases)
        else:
            # Try to find the field "as is" in the file
            try:
                field_arr = radar_obj.fields[field]
                field_to_plot = (field, field_arr)
            except KeyError:
                field_to_plot = None
        if field_to_plot:
            fields_to_plot.append(field_to_plot[0])

        if plot_filtered:
            if field in REF_aliases:
                field_to_plot = get_field_to_plot(radar_obj, REF_aliases, tag='_filtered')
            elif field in ZDR_aliases:
                field_to_plot = get_field_to_plot(radar_obj, ZDR_aliases, tag='_filtered')
            elif field in RHV_aliases:
                field_to_plot = get_field_to_plot(radar_obj, RHV_aliases, tag='_filtered')
            if field_to_plot:
                fields_to_plot.append(field_to_plot[0])

    for field in fields_to_plot:
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': projection})
        titlestringfmt = 'Radar name: {}; Field: {}; Time: {}; elevation: {:2.1f}'
        titlestring = titlestringfmt.format(radar_obj.metadata['instrument_name'], field,
                                            sweeptime.strftime(tm.timefmt3),
                                            radar_obj.elevation['data'][0])

        field_to_match = field
        # DTD: find a better way to do this
        retr_aliases = [retr_alias for aliases in retr_alias_list for retr_alias in aliases]
        for field_trial in retr_aliases:
            if field_trial in field:
                field_to_match = field_trial
                break
        field_to_match = field_to_match.replace('_filtered', '')
        field_plot_params = radar_plot_param_matching[field_to_match]

        # TODO: add colorbar label levels, other arguments
        
        var = radar_obj.fields[field]['data']
        
        if norm is None:
            ci = ax.pcolormesh(xplt, yplt, var.squeeze(), vmin=field_plot_params['clevels'][0], 
                               vmax=field_plot_params['clevels'][-1], cmap=field_plot_params['cmap'], 
                               norm=norm)
        else:
            ci = ax.pcolormesh(xplt, yplt, var.squeeze(), cmap=field_plot_params['cmap'], norm=norm)
        
        ax.set_title(titlestring, fontsize=10)
        # Overlay locations of the PIPS
            
        for PIPS_name in PIPS_names:
            # PIPS_names_toplot.append(PIPS_name)
            # rad_locs_toplot.append(rad_loc_dict[PIPS_name])
            PIPS_x = PIPS_x_dict[PIPS_name]
            PIPS_y = PIPS_y_dict[PIPS_name]
            ax.plot([PIPS_x], [PIPS_y], 'r*', ms=10, alpha=0.5)
            x_text = text_x_dict[PIPS_name]
            y_text = text_y_dict[PIPS_name]
            ax.annotate(PIPS_name, xy=(x_text, y_text))

        # Overlay roads
        ax.add_feature(cartopy.feature.NaturalEarthFeature('cultural', 'roads', '10m'),
                       facecolor='none', edgecolor='black', alpha=.3)
            
        if isinstance(field_plot_params['cbint'], list):
            cbarlevels = field_plot_params['cbint']
        else:
            cbarintv = field_plot_params['cbint'] # field_plot_params['clevels'][1] - field_plot_params['clevels'][0]
            cbarlevels = ticker.MultipleLocator(base=cbarintv)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05, axes_class=matplotlib.axes.Axes)
        fig.colorbar(ci, orientation='vertical', ticks=cbarlevels, cax=cax)
        if cbarlabel is not None:
            cax.set_ylabel(cbarlabel)
#         formatter = ticker.FuncFormatter(mtokm)
#         ax.xaxis.set_major_formatter(formatter)
#         ax.yaxis.set_major_formatter(formatter)
#         ax.xaxis.set_major_locator(ticker.MultipleLocator(base=axestickintv))
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(base=axestickintv))
#         ax.set_xlabel('km')
#         ax.set_ylabel('km')
#         ax.set_xlim(xmin, xmax)
#         ax.set_ylim(ymin, ymax)
#         ax.set_aspect('equal')

        gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, color = "None", 
                          crs=ccrs.PlateCarree(), rotate_labels=False)
        gl.xlocator = cartopy.mpl.ticker.LongitudeLocator() # ticker.FixedLocator(np.arange(-110, -75, 0.1))
        gl.ylocator = cartopy.mpl.ticker.LatitudeLocator() # ticker.FixedLocator(np.arange(30, 46, 0.1))
        gl.xformatter = cartopy.mpl.ticker.LongitudeFormatter()
        gl.yformatter = cartopy.mpl.ticker.LatitudeFormatter()
        gl.bottom_labels = True
        gl.left_labels   = True
        gl.top_labels    = False
        gl.right_labels  = False
        ax.set_extent([xmin, xmax, ymin, ymax], crs=projection)
        
#         ax.xaxis.set_major_formatter(cartopy.mpl.gridliner.LONGITUDE_FORMATTER)
#         ax.yaxis.set_major_formatter(cartopy.mpl.gridliner.LATITUDE_FORMATTER)

        figlist.append(fig)
        axlist.append(ax)

    return figlist, axlist, fields_to_plot


In [ ]:
for i, radar_path in enumerate(radar_path_dict['rad_path_list']):
    print(i, radar_path)

In [ ]:
for radar_path in radar_path_dict['rad_path_list'][22:23]:
    radar_obj_list = radar.readCFRadial_pyART(el_req, radar_path, compute_kdp=False)
    # Loop through the matching sweeps in the file (there may be more than one because of
    # split cuts/SAILS)
    for radar_obj in radar_obj_list[:1]:
        # Extract time from start of sweep
        sweep_time = pyart.graph.common.generate_radar_time_sweep(radar_obj, 0)
        # Convert to np.datetime64
        sweep_time_dt64 = np.datetime64(sweep_time)
        sweep_time_string = sweep_time.strftime(tm.timefmt3)
        
        # Figure out which PIPS (if any) were deployed at this radar time and add them to the lists to plot
        PIPS_names_toplot = []
        geo_locs_toplot = []
        rad_locs_toplot = []
        geo_locs_dict_toplot = {}
        rad_locs_dict_toplot = {}
        
        for PIPS_name, PIPS_ds in PIPS_ds_dict.items():
            PIPS_start_time = PIPS_ds.time[0].values
            PIPS_end_time = PIPS_ds.time[-1].values
            
            if sweep_time_dt64 >= PIPS_start_time and sweep_time_dt64 <= PIPS_end_time:
                PIPS_names_toplot.append(PIPS_name)
                geo_locs_toplot.append(geo_loc_dict[PIPS_name])
                rad_locs_toplot.append(rad_loc_dict[PIPS_name])
                geo_locs_dict_toplot[PIPS_name] = geo_loc_dict[PIPS_name]
                rad_locs_dict_toplot[PIPS_name] = rad_loc_dict[PIPS_name]
        
        print(sweep_time_string)
        
        figlist, axlist, fields_plotted = plotsweep_pcolor(radar_obj, field_names, sweep_time, 
                                                           PIPS_names=PIPS_names_toplot,
                                                           PIPS_rad_loc_dict=rad_locs_dict_toplot,
                                                           plot_filtered=True, bounds=bounds, axestickintv=1000.)
        
        
#         figlist, axlist, fields_plotted = plotsweep_pyART(radar_obj, sweep_time, PIPS_names_toplot,
#                                                           geo_locs_toplot, rad_locs_toplot, field_names,
#                                                           plot_filtered=True, bounds=bounds)

In [ ]:
print(cartopy.__version__)

In [ ]:
import cartopy.mpl.gridliner

In [ ]:
sweep_time_datetime64 = np.datetime64(sweep_time)

In [ ]:
type(sweep_time_datetime64)